In [11]:
import cv2
import numpy as np

video_path = r"C:\Users\NIHAL_MIRAJ\Desktop\computer vission\Computer Vision Projects\36510-411342239_small.mp4"
cap = cv2.VideoCapture(video_path)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # codec for mp4
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))


out_path = r"C:\Users\NIHAL_MIRAJ\Desktop\computer vission\Computer Vision Projects\output_tracking.mp4"
out = cv2.VideoWriter(out_path, fourcc, fps, (width, height))


ret, frame = cap.read()
if not ret:
    print("Cannot read video file.")
    cap.release()
    exit()

r = cv2.selectROI("Select ROI", frame, fromCenter=False, showCrosshair=True)
x, y, w, h = r
track_window = (x, y, w, h)

roi = frame[y:y+h, x:x+w]
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60., 32.)), np.array((180., 255., 255.)))

roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
    ret, track_window = cv2.CamShift(dst, track_window, term_crit)

    pts = cv2.boxPoints(ret)
    pts = np.int0(pts)
    final = cv2.polylines(frame, [pts], True, (0, 255, 0), 2)
    cv2.imshow("CAMShift CCTV Tracking", final)

    out.write(final)

    key = cv2.waitKey(30) & 0xFF
    if key == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

C:\Users\NIHAL_MIRAJ\AppData\Local\Temp\ipykernel_9280\3411186408.py:46: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  pts = np.int0(pts)


✅ Tracking video saved at: C:\Users\NIHAL_MIRAJ\Desktop\computer vission\Computer Vision Projects\output_tracking.mp4
